# Atelier pratique : Comprendre l’IA par la pratique
*Version mise à jour – 20 May 2025*

Ce notebook a été pensé pour des **débutants complets** : chaque cellule de code est abondamment commentée et peut être exécutée pas à pas.  
S’il s’agit de votre **premier contact avec Python**, suivez simplement ces trois règles :

1. Cliquez sur la cellule (elle est entourée d’un cadre bleu).  
2. Appuyez sur le bouton ▶️ (*Run*) ou sur **Shift + Enter** pour l’exécuter.  
3. Observez le résultat qui apparaît juste en dessous.

> **Important :** si vous exécutez ce notebook **localement** (VS Code, Jupyter…), assurez‑vous d’abord d’installer les librairies nécessaires :  
> `pip install scikit-learn matplotlib numpy tensorflow-cpu transformers`

---


## 1️⃣ Classification supervisée – jeu de données *Iris*

In [ ]:
# 🔶 Étape 1 : importations
# La librairie scikit‑learn contient des jeux de données et des algorithmes "prêt à l'emploi".
from sklearn import datasets                       # jeux de données jouets
from sklearn.model_selection import train_test_split # découpe aléatoire train/test
from sklearn.linear_model import LogisticRegression  # algorithme de régression logistique
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt                     # affichage de graphes
import numpy as np                                  # calcul numérique

# 🔶 Étape 2 : chargement du jeu de données
iris = datasets.load_iris()  # 150 fleurs, 4 mesures, 3 espèces
X, y = iris.data, iris.target  # X = caractéristiques, y = étiquettes

print(f"Taille du jeu de données : {X.shape[0]} échantillons × {X.shape[1]} caractéristiques")

# 🔶 Étape 3 : séparation en apprentissage (75 %) et test (25 %)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y)  # stratify garantit les mêmes proportions d'espèces

# 🔶 Étape 4 : création et entraînement du modèle
clf = LogisticRegression(max_iter=200)  # 200 itérations max
clf.fit(X_train, y_train)               # "apprendre" = ajuster les paramètres sur les données d'entraînement

# 🔶 Étape 5 : évaluation sur les données jamais vues (test)
y_pred = clf.predict(X_test)            # prédiction de l'espèce
acc = accuracy_score(y_test, y_pred)    # proportion de bonnes réponses
print(f"Précision (accuracy) : {acc:.2f}")

# 🔶 Étape 6 : matrice de confusion (visualise les erreurs)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='Blues')
plt.title('Matrice de confusion – Iris')
plt.show()


### ➡️ À tester  
1. Changez `test_size` à **0.4** ou **0.1** pour observer l’impact sur la précision.  
2. Remplacez `LogisticRegression` par `DecisionTreeClassifier` (à importer depuis `sklearn.tree`) puis ré‑exécutez.  
3. Modifiez `random_state` (graine aléatoire) : le découpage change, la précision aussi.  


## 2️⃣ Réseau de neurones de base avec Keras/TensorFlow

In [ ]:
# 🧠 Importations
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# 🧠 Définition de l'architecture du réseau
# - input_shape=(4,) car chaque fleur = 4 chiffres
# - 1 couche cachée de 16 neurones (Dense) avec fonction d'activation ReLU
# - sortie Dense(3) car 3 espèces, activation softmax -> probas qui se somment à 1
model = models.Sequential([
    layers.Dense(16, activation='relu', input_shape=(4,)),
    layers.Dense(3, activation='softmax')
])

# 🧠 Compilation : on choisit comment apprendre
model.compile(
    optimizer='adam',                          # méthode de descente de gradient
    loss='sparse_categorical_crossentropy',    # adéquat pour plusieurs classes
    metrics=['accuracy']                       # on suit l'accuracy
)

# 🧠 Apprentissage
history = model.fit(
    X_train, y_train,
    epochs=50,             # 50 passes sur les données
    batch_size=16,         # nombre d'exemples vus avant mise à jour des poids
    verbose=0,             # 0 = silencieux, 1 = barre de progression
    validation_split=0.2   # 20 % du train sert à valider (détecter overfitting)
)

# 🧠 Visualisation de la courbe d'apprentissage
plt.plot(history.history['accuracy'], label='Train acc')
plt.plot(history.history['val_accuracy'], label='Validation acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Apprentissage du réseau de neurones')
plt.show()

# 🧠 Évaluation finale sur le jeu de test
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Précision sur le test : {test_acc:.2f}")


### ➡️ À tester  
* **Complexité du réseau :** ajoutez une seconde couche Dense, ou augmentez le nombre d’unités (ex : 32).  
* **Fonction d’activation :** essayez `tanh` ou `elu` à la place de `relu`.  
* **Nombre d’epochs :** mettez 200 et observez si le modèle **sur‑apprend** (la courbe validation diverge).  


## 3️⃣ Génération de texte (LLM open source GPT‑2)

In [ ]:
# ✍️ Importer un pipeline de génération depuis HuggingFace Transformers
from transformers import pipeline

# Le modèle GPT‑2 (124 M paramètres) tient dans 500 MB – raisonnable pour un atelier
generator = pipeline('text-generation', model='gpt2')

prompt = "Dans le futur,"  # phrase de départ
resultats = generator(prompt, max_length=40, num_return_sequences=1)

print("\n--- Texte généré ---\n")
print(resultats[0]['generated_text'])


> **Note sécurité/éthique :** GPT‑2 est open‑source et non filtré ; il peut produire du contenu inexact ou inapproprié. N’utilisez jamais les sorties sans relecture humaine.  


## 4️⃣ (Bonus) Biais et équité – mini‑simulation

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score

# 🎲 Générons un jeu de données *biaisé*
np.random.seed(0)
n = 200

# sensitive_attribute = appartenance à un groupe (0 ou 1)
sensitive_attribute = np.random.randint(0, 2, size=n)

# feature = caractéristique 'méritocratique'
feature = np.random.randn(n)

# label = décision d'embauche (1) ou refus (0); 
# le groupe 1 a +0.5 point d'avantage -> biais dans la *réalité*
label = (feature + sensitive_attribute * 0.5 > 0).astype(int)

X = feature.reshape(-1, 1)
y = label

# ⚖️ Modèle naïf qui ignore la variable sensible
clf = LogisticRegression().fit(X, y)
pred = clf.predict(X)

# ⚖️ Équité : comparons la balanced_accuracy par groupe
for group in [0, 1]:
    idx = sensitive_attribute == group
    acc = balanced_accuracy_score(y[idx], pred[idx])
    print(f"Groupe {group} – balanced accuracy : {acc:.2f}")


Vous devriez constater que le modèle **n’efface pas** le biais d’origine ; au contraire il peut le renforcer.  
👉 Conclusion : évaluer un modèle seulement sur une métrique globale peut masquer des discriminations.  
